# 프로젝트 Movielnes 영화 추천실습
   
     
        
        

``` wget http://files.grouplens.org/datasets/movielens/ml-1m.zip```

## libaryry import

In [2]:
import os
import pandas as pd 
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## data improt


In [3]:
#rating_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/ratings.dat'
rating_file_path= './data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## data preprocessing

In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)



print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)


In [6]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [7]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'count']
data = ratings[using_cols]
data.tail(10)

,user_id,movie_id,count
1000198,6040,2021,3
1000199,6040,2022,5
1000200,6040,2028,5
1000201,6040,1080,4
1000202,6040,1089,4
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
#movie_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/movies.dat'
movie_file_path= './data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.tail(10)

,movie_id,title,genre
3873,3943,Bamboozled (2000),Comedy
3874,3944,Bootmen (2000),Comedy|Drama
3875,3945,Digimon: The Movie (2000),Adventure|Animation|Children's
3876,3946,Get Carter (2000),Action|Drama|Thriller
3877,3947,Get Carter (1971),Thriller
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [9]:
# 유저 이름을 보기 위해 메타 데이터를 읽어옵니다.
#movie_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/movies.dat'
users_file_path= './data/ml-1m/users.dat'
users_cols = ['user_id', 'gender', 'age','occupation','Zip-code' ] 
users = pd.read_csv(users_file_path, sep='::', names=users_cols, engine='python')
users.tail(10)

,user_id,gender,age,occupation,Zip-code
6030,6031,F,18,0,45123
6031,6032,M,45,7,55108
6032,6033,M,50,13,78232
6033,6034,M,25,14,94117
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [10]:
#user_unique = data['user_id'].unique()
movie_names = movies[['movie_id','title']]
movie_names

#movie_title = {val:idx for idx, val in movies['movie_id'].data,movies['title'].data}
# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
#user_to_idx = {v:k for k,v in users.items()}
#movie_to_idx = {v:k for k,v in  enumerate(artist_unique)}

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
3878,3948,Meet the Parents (2000)
3879,3949,Requiem for a Dream (2000)
3880,3950,Tigerland (2000)
3881,3951,Two Family House (2000)


In [11]:
ratings.nunique()

user_id        6039
movie_id       3628
count             3
timestamp    412911
dtype: int64

## data <분석
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개(인기순)

In [12]:
# 영화 개수
data['movie_id'].nunique()

3628

In [13]:
# 사용자 수
data['user_id'].nunique()

6039

In [14]:
# 가장 인기 있는 영화 30개 인기수
#TODO movie id 와 movies title 매핑
movie_count= data.groupby('movie_id')['user_id'].count()
#movie_count['movie_id'].map(movies.get).dropna()
#print(movies[movies['movie_id'] == 2858]['title'])   movies[movie_count['movie_id'] == movies['movie_id']]['title']
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [15]:
print(movies[movies['movie_id'] == 2858]['title']) 

2789    American Beauty (1999)
Name: title, dtype: object


## data add 
- 내가 선호하는 영화 5가지 골라서 rating 추가


In [16]:
movies[movies["genre"]== "Action"].head(10)
#movies[movies["genre"]== "Drama"].head(30)

,movie_id,title,genre
8,9,Sudden Death (1995),Action
19,20,Money Train (1995),Action
70,71,Fair Game (1995),Action
143,145,Bad Boys (1995),Action
202,204,Under Siege 2: Dark Territory (1995),Action
224,227,Drop Zone (1994),Action
248,251,"Hunted, The (1995)",Action
312,315,"Specialist, The (1994)",Action
380,384,Bad Company (1995),Action
389,393,Street Fighter (1994),Action


In [17]:

movies[movies['title']=='Bad Company (1995)']['movie_id']


380    384
Name: movie_id, dtype: int64

In [18]:
my_favorite = ['Bad Company (1995)' , 
               'Street Fighter (1994)' ,
               'Bad Boys (1995)' ,
               'Sudden Death (1995)' ,
               'Fair Game (1995)']
my_favorite_id= [384, 393, 145, 9, 71 ]

# id --> title조회
for m in my_favorite_id:
    a = movies[movies['movie_id']==m]['title']
    print(np.array(a.tolist())[0])

print('\n')

# title --> id 조회
for m in my_favorite:
    a = movies[movies['title']==m]['movie_id']
    print(np.array(a.tolist())[0])

Bad Company (1995)
Street Fighter (1994)
Bad Boys (1995)
Sudden Death (1995)
Fair Game (1995)


384
393
145
9
71


In [19]:
data

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [20]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [21]:
 


my_playlist = pd.DataFrame({'user_id': [6041]*5, 
                            'movie_id':my_favorite_id, 
                            'count':4})

if not data.isin({'user_id':['6041']})['user_id'].any(): 
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
#data = data.append(my_playlist)     
data.tail(10)

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,384,4
1,6041,393,4
2,6041,145,4
3,6041,9,4
4,6041,71,4


In [22]:
data[["user_id","movie_id","count"]].to_csv("test2.csv")

In [23]:
data['user_id'].nunique()
# 1명 추가 됨...

6040

In [24]:
data.nunique()

user_id     6040
movie_id    3628
count          3
dtype: int64

In [25]:
#data['user_id'].isnull().values.sum()
#data['movie_id'].isnull().values.sum()
data.isnull().sum()

user_id     0
movie_id    0
count       0
dtype: int64

## csr matrix 

In [26]:
#from scipy.sparse import csr_matrix


num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()





In [27]:
#index ####################################################################

#고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

# 유저, 영화 indexing 하는 코드.
user_to_idx = {v : k for k, v in enumerate(user_unique)}
movie_to_idx = {v : k for k, v in enumerate(movie_unique)}

# 인덱싱 제대로 되었는지 확인
print(user_to_idx[6039])

temp_user_data = data['user_id'].map(user_to_idx.get)#.dropna()

if len(temp_user_data) == len(data):
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data
else:
    print('user_id column indexing fail')
    
temp_movie_data = data['movie_id'].map(movie_to_idx.get)#.dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing ok!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie column indexing fail')

data.tail(10)




6037
user_id column indexing OK!!
movie column indexing ok!!


,user_id,movie_id,count
1000203,6038,1030,3
1000205,6038,986,5
1000206,6038,311,5
1000207,6038,142,4
1000208,6038,26,4
0,6039,2256,4
1,6039,2115,4
2,6039,2054,4
3,6039,1941,4
4,6039,2109,4


In [28]:
#
for idx in enumerate(movie_to_idx):
    if idx == movie_to_idx.get(idx):
        print (idx)

In [149]:
print(num_user, num_movie)
# print(data2['count'].shape)

# print(type(data))
print(type(num_user))
# print(type(data2.user_id))
# print(type(data2['count']))
csr_data = csr_matrix((data['count'], (data.user_id, data.movie_id)), shape= (num_user, num_movie)) 
#csr_data = csr_matrix((data['count'], (data.user_id, data.movie_id)), shape= (6050, 3955))
csr_data

6040 3628
<class 'int'>


<6050x3955 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [150]:
data.nunique()

user_id     6040
movie_id    3628
count          3
dtype: int64

## als_model = AlternatingLeastSquares 모델 훈련

In [151]:
# Implicit AlternatingLeastSquares 모델의 선언
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [152]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3955x6050 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [153]:
# 모델 훈련
als_model.fit(csr_data_transpose)

## 선호도 파악

In [154]:
# 나의 선호도
me_idx = 6041
me_vector  = als_model.user_factors[me_idx]
print(me_vector)


[-0.05710668  0.22750214  0.2722711   0.03485942  0.14750381  0.4164791
 -0.19536044 -0.190876    0.26535752 -0.28242546 -0.12498345 -0.4197336
  0.21837421  0.31551751  0.1394324   0.04765859  0.16862766 -0.18272439
  0.09028581 -0.09651048 -0.20890188  0.1396397  -0.23149715 -0.10154201
 -0.3941115  -0.14933783 -0.1631806  -0.0859261  -0.26226723 -0.24894582
 -0.31982568 -0.2983636   0.48518836 -0.14566465  0.0901081  -0.00900921
 -0.09509514 -0.33249992 -0.18580529 -0.28945425  0.14716382 -0.09902316
  0.250647    0.14332214  0.5242003   0.28738865  0.32168564 -0.03431356
  0.04538621  0.31282684  0.09055667 -0.0474662   0.24868783 -0.09329617
  0.02534539  0.19777808 -0.04706057  0.15038654  0.01642139  0.12044958
 -0.01699697  0.4931224  -0.05015451 -0.17842464 -0.03999742  0.17153154
  0.08245417 -0.11186646  0.14121546  0.02940539 -0.27041373  0.03732961
  0.04665237 -0.1821335   0.10630115 -0.1845696  -0.14185382 -0.127251
 -0.0641429   0.03323484 -0.7650244  -0.03469595 -0.163

In [155]:
#'Fair Game (1995)'  idx 71 선호도   movie_names
fg_idx = 71
fg_vector = als_model.item_factors[fg_idx]
print(fg_vector)

np.dot(me_vector, fg_vector)

[ 2.16249912e-03  6.51787315e-03  7.96318147e-03  5.72506338e-03
  6.85767783e-03  6.79405918e-03  2.75673950e-03  3.69561999e-03
  1.12469215e-02 -6.80002943e-03  2.89966865e-03 -4.25504567e-03
  4.46897605e-03  9.71207116e-03  5.32678934e-03  3.33890319e-03
  9.04870126e-03  7.34148023e-04  4.94223367e-03  1.96936168e-03
 -5.59547078e-03  9.53859650e-03 -4.50107874e-03 -6.40060636e-04
 -3.17147584e-03  2.17674766e-04  4.34645079e-03 -1.48398531e-05
 -4.64025798e-04  2.55532097e-03 -1.73040270e-03 -2.26214644e-03
  9.24530160e-03  3.13587044e-03  9.75881424e-03  3.54054011e-03
  1.19706721e-03 -4.08997666e-03 -1.36903406e-03 -3.18350765e-04
  7.40229012e-03  3.24121793e-03  7.88124092e-03  8.40932317e-03
  6.43409509e-03  9.55925044e-03  6.67330809e-03 -2.47083459e-04
  1.86836021e-03  7.38879433e-03  1.81538588e-03  6.02815766e-03
  2.04537273e-03  4.19569341e-03  1.48406171e-03  1.16649922e-02
  2.56033032e-03  2.49447301e-03  3.05926194e-03  6.56989543e-03
  3.15997377e-03  9.28951

0.075593196

In [156]:
#'Street Fighter (1994)' , idx 393 선호도   movie_names
sf_idx = 393
sf_vector = als_model.item_factors[sf_idx]
print(sf_vector)

np.dot(me_vector, sf_vector)


[ 0.00093874  0.00734859  0.00277833  0.00159643  0.0055576   0.00747156
  0.00054147  0.00278453  0.00549914 -0.00703289  0.00308854 -0.00827643
  0.00321859  0.00645012  0.00251858  0.0034711   0.00629672 -0.00686235
  0.00425294 -0.00022205 -0.002049    0.00072898 -0.00368968  0.00287623
  0.00349327  0.0002234   0.00464785 -0.00221567 -0.00375902 -0.00265387
 -0.00290103 -0.00068598  0.00849719  0.00256318  0.00787104  0.00161155
  0.00022458 -0.00247918 -0.00127723  0.00333749  0.00454649  0.00055018
  0.00171565  0.00030839  0.01155572  0.00229516  0.00919427 -0.00042413
  0.00148278  0.00627162  0.00499428 -0.00249652 -0.00198571  0.00452554
 -0.00164888  0.00114933  0.00948388  0.001827    0.00358298  0.0100385
  0.00319368  0.00420433  0.00092989  0.00345489  0.00668054  0.00535919
 -0.00061639  0.0035068   0.0009507   0.00199616 -0.00308597 -0.00168376
  0.00212234  0.00418817  0.00706855  0.00084256 -0.00140264  0.00085571
  0.00186892  0.00321782 -0.00369159 -0.00111578  0.

0.052702717

In [157]:
#'Bad Boys' , idx 393 선호도   movie_names
bb_idx = 145
bb_vector = als_model.item_factors[bb_idx]
print(bb_vector)

np.dot(me_vector, bb_vector)
# my_favorite = ['Bad Company (1995)' , 
#                'Street Fighter (1994)' ,
#                'Bad Boys' ,
#                'Sudden Death (1995)' ,
#                'Fair Game (1995)']
# my_favorite_id= [384, 393, 145, 9, 71 ]

[ 2.32755439e-03  8.39417148e-03  2.60543567e-03  9.97536257e-03
  4.76756878e-03  3.55004221e-02 -2.13647517e-03 -2.12619896e-03
  1.43300975e-02  6.05126191e-03 -2.81346584e-04  1.64092577e-03
  9.57981683e-03  9.37282946e-03 -1.67393161e-03  5.77872479e-03
  2.15783686e-04  6.87429701e-06  7.58644380e-03 -4.71430685e-04
 -6.51970832e-03  1.35070831e-02  4.32438712e-04  2.53723422e-03
 -1.20627573e-02 -2.54196092e-03 -1.58593375e-02  8.28288123e-03
 -3.42318392e-03 -5.42579079e-03 -6.46894518e-03  2.51105218e-03
  2.61445791e-02  1.79670204e-03  5.24540991e-03 -4.55267320e-04
 -8.38393695e-04 -5.96646406e-03 -1.40377060e-02 -8.30661133e-03
  8.26576352e-03  8.26054905e-03  1.88006405e-02  3.60372523e-03
  2.26271581e-02  7.76474597e-03  1.71331894e-02  2.05242112e-02
  8.87964386e-03  1.38467969e-02  9.10131447e-03  4.99355432e-04
  1.17477933e-02 -1.51719963e-02  1.46522038e-02  5.74465189e-03
  1.62627688e-03  2.58269031e-02 -1.99094275e-03 -7.73148425e-03
  6.50695246e-03  2.41055

0.17437297

## 유사 영화 추천

In [158]:
#'Fair Game (1995)'  idx 71 선호도   movie_names
fg_idx = 71
similar_movies = als_model.similar_items(fg_idx, N=15)
similar_movies

[(71, 0.056457765),
 (544, 0.048570935),
 (1744, 0.04845251),
 (548, 0.048448354),
 (1626, 0.048133466),
 (1004, 0.04812112),
 (20, 0.046849713),
 (1497, 0.04676573),
 (3165, 0.046566322),
 (1869, 0.04650586),
 (990, 0.046472967),
 (694, 0.046243593),
 (1604, 0.046098005),
 (2835, 0.046009857),
 (886, 0.045923125)]

In [159]:
# title 매칭
for m in similar_movies:
    print(movies[movies['movie_id']==m[0]][['title','genre']])

               title   genre
70  Fair Game (1995)  Action
                        title   genre
540  Striking Distance (1993)  Action
                 title                      genre
1693  Firestorm (1998)  Action|Adventure|Thriller
                        title   genre
544  Terminal Velocity (1994)  Action
                       title                  genre
1583  Fire Down Below (1997)  Action|Drama|Thriller
                       title            genre
991  Glimmer Man, The (1996)  Action|Thriller
                 title   genre
19  Money Train (1995)  Action
                   title   genre
1465  Double Team (1997)  Action
                     title         genre
3096  Boiling Point (1993)  Action|Drama
                 title            genre
1800  Black Dog (1998)  Action|Thriller
                   title                      genre
978  Maximum Risk (1996)  Action|Adventure|Thriller
                      title   genre
685  Substitute, The (1996)  Action
                   title    

## 좋아할 만한 영화 추천

In [160]:
#  
me_idx = 6041
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(me_idx, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(204, 0.10826999),
 (434, 0.103550516),
 (20, 0.09599974),
 (44, 0.094451934),
 (1385, 0.09306033),
 (464, 0.092892736),
 (886, 0.09147255),
 (1769, 0.091011696),
 (1866, 0.09079138),
 (548, 0.09023849),
 (423, 0.0898376),
 (1004, 0.08698315),
 (2808, 0.085617915),
 (315, 0.08448044),
 (996, 0.08430076),
 (1382, 0.08364885),
 (511, 0.082483545),
 (694, 0.08188057),
 (3444, 0.081383266),
 (1047, 0.08135515)]

In [161]:
## 변환
a = []
# title 
for m in movie_recommended:
    a.append(movies[movies['movie_id']==m[0]]['title'])
    print(movies[movies['movie_id']==m[0]][['title','genre']])

                                    title   genre
202  Under Siege 2: Dark Territory (1995)  Action
                  title                   genre
430  Cliffhanger (1993)  Action|Adventure|Crime
                 title   genre
19  Money Train (1995)  Action
                   title             genre
43  Mortal Kombat (1995)  Action|Adventure
                   title   genre
1364  Under Siege (1992)  Action
                  title                            genre
460  Hard Target (1993)  Action|Adventure|Crime|Thriller
                  title   genre
874  Bulletproof (1996)  Action
                                title            genre
1713  Replacement Killers, The (1998)  Action|Thriller
                    title          genre
1797  Big Hit, The (1998)  Action|Comedy
                        title   genre
544  Terminal Velocity (1994)  Action
                 title            genre
419  Blown Away (1994)  Action|Thriller
                       title            genre
991  Glimmer Man, 

## 코멘트....
- 주입한 데이터를 액션영화 5개만을 골라서 입력해서 그런지 유사 영화와 추천영화의 장르가 전부 액션으로 나왔다.